In [1]:
import sys
import os
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency, ttest_ind
import matplotlib.pyplot as plt
import seaborn as sns
sys.path.append('../scripts')

In [2]:
# Load the original .txt file using the correct separator
df = pd.read_csv('../data/MachineLearningRating_v3_cleaned.csv')

C:\Users\amanu\AppData\Local\Temp\ipykernel_18728\1578795113.py:2: DtypeWarning: Columns (45) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/MachineLearningRating_v3_cleaned.csv')


### Data Cleaning and Preparation

In [3]:
# Check for missing values in each column
missing_values = df.isna().sum()
print("Missing values per column:")
print(missing_values)

# Check the percentage of missing values
missing_percentage = (missing_values / len(df)) * 100
print("\nPercentage of missing values per column:")
print(missing_percentage)


Missing values per column:
,UnderwrittenCoverID        0
PolicyID                    0
TransactionMonth            0
IsVATRegistered             0
Citizenship                 0
LegalType                   0
Title                       0
Language                    0
Bank                        0
AccountType                 0
MaritalStatus               0
Gender                      0
Country                     0
Province                    0
PostalCode                  0
MainCrestaZone              0
SubCrestaZone               0
ItemType                    0
mmcode                      0
VehicleType                 0
RegistrationYear            0
make                        0
Model                       0
Cylinders                   0
cubiccapacity               0
kilowatts                   0
bodytype                    0
NumberOfDoors               0
VehicleIntroDate            0
AlarmImmobiliser            0
TrackingDevice              0
CapitalOutstanding          0
NewVehicle   

In [5]:
# Verify if all missing values are handled
print(df.isna().sum())


,UnderwrittenCoverID        0
PolicyID                    0
TransactionMonth            0
IsVATRegistered             0
Citizenship                 0
LegalType                   0
Title                       0
Language                    0
Bank                        0
AccountType                 0
MaritalStatus               0
Gender                      0
Country                     0
Province                    0
PostalCode                  0
MainCrestaZone              0
SubCrestaZone               0
ItemType                    0
mmcode                      0
VehicleType                 0
RegistrationYear            0
make                        0
Model                       0
Cylinders                   0
cubiccapacity               0
kilowatts                   0
bodytype                    0
NumberOfDoors               0
VehicleIntroDate            0
AlarmImmobiliser            0
TrackingDevice              0
CapitalOutstanding          0
NewVehicle                  0
SumInsured

In [16]:
numeric_columns = df.select_dtypes(include=[np.number]).columns
print("Numerical columns:", numeric_columns)
def identify_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers

for column in numeric_columns:
    outliers = identify_outliers(df, column)
    print(f"\nOutliers in {column}:")
    print(f"Number of outliers: {len(outliers)}")
    print(f"Percentage of outliers: {(len(outliers) / len(df)) * 100:.2f}%")
    print(outliers[column].describe())
    print("\n" + "-"*50)

Numerical columns: Index(['PolicyID', 'PostalCode', 'mmcode', 'RegistrationYear', 'Cylinders',
       'cubiccapacity', 'kilowatts', 'NumberOfDoors', 'CapitalOutstanding',
       'SumInsured', 'CalculatedPremiumPerTerm', 'TotalPremium',
       'TotalClaims'],
      dtype='object')

Outliers in PolicyID:
Number of outliers: 31232
Percentage of outliers: 3.12%
count    31232.000000
mean     21730.691246
std        425.654385
min      20943.000000
25%      21353.000000
50%      21648.500000
75%      22223.000000
max      22223.000000
Name: PolicyID, dtype: float64

--------------------------------------------------

Outliers in PostalCode:
Number of outliers: 0
Percentage of outliers: 0.00%
count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: PostalCode, dtype: float64

--------------------------------------------------

Outliers in mmcode:
Number of outliers: 241622
Percentage of outliers: 24.16%
count    2.416220e+05
mean     3.865

In [22]:
columns_to_clean = ['cubiccapacity', 'kilowatts', 'CapitalOutstanding', 'SumInsured', 'CalculatedPremiumPerTerm', 'TotalPremium', 'TotalClaims']
def cap_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df[column] = df[column].clip(lower_bound, upper_bound)
    return df

df_capped = df.copy()
for col in columns_to_clean:
    df_capped = cap_outliers(df_capped, col)

In [24]:
columns_to_clean = ['cubiccapacity', 'kilowatts', 'CapitalOutstanding', 'SumInsured', 'CalculatedPremiumPerTerm', 'TotalPremium', 'TotalClaims']

# Function to identify outliers
def identify_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers

# Function to cap outliers (Winsorization)
def cap_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df[column] = df[column].clip(lower_bound, upper_bound)
    return df

# Create cleaned dataframe by capping outliers
df_cleaned = df.copy()
for column in columns_to_clean:
    df_cleaned = cap_outliers(df_cleaned, column)

In [25]:
def check_remaining_outliers(df, columns):
    for column in columns:
        outliers = identify_outliers(df, column)
        print(f"\nRemaining outliers in {column}:")
        print(f"Number of outliers: {len(outliers)}")
        print(f"Percentage of outliers: {(len(outliers) / len(df)) * 100:.2f}%")
        if len(outliers) > 0:
            print(outliers[column].describe())
        print("-" * 50)

check_remaining_outliers(df_cleaned, columns_to_clean)


Remaining outliers in cubiccapacity:
Number of outliers: 0
Percentage of outliers: 0.00%
--------------------------------------------------

Remaining outliers in kilowatts:
Number of outliers: 0
Percentage of outliers: 0.00%
--------------------------------------------------

Remaining outliers in CapitalOutstanding:
Number of outliers: 0
Percentage of outliers: 0.00%
--------------------------------------------------

Remaining outliers in SumInsured:
Number of outliers: 0
Percentage of outliers: 0.00%
--------------------------------------------------

Remaining outliers in CalculatedPremiumPerTerm:
Number of outliers: 0
Percentage of outliers: 0.00%
--------------------------------------------------

Remaining outliers in TotalPremium:
Number of outliers: 0
Percentage of outliers: 0.00%
--------------------------------------------------

Remaining outliers in TotalClaims:
Number of outliers: 0
Percentage of outliers: 0.00%
--------------------------------------------------


### Data Segmentation 

In [32]:
# Segment data into Group A (Control Group) and Group B (Test Group) for Provinces
groupA = df[df['Province'] == 'Gauteng']  # Replace with actual province name
groupB = df[df['Province'] == 'Western Cape']  # Replace with actual province name

# Optionally, check for statistical equivalence between groups on other features


In [33]:
# Check unique values in the Gender column
unique_genders = df['Gender'].unique()
print(unique_genders)


['Not specified' 'Male' 'Female']


In [34]:
# Check unique values in the PostalCode column
unique_postal_codes = df['PostalCode'].unique()
print(unique_postal_codes)


[1459 1513 1619 1625 1629 1852 1982 2007 2066 4093 2000 1577 1610 2410
  122 1520 1709 1739 4000 4091 4342 4359 7784  970 6213 6390 1868 4310
  309  152  181 1821 4449 4037  139 4074 1057 7100 8566 1863 1875 2001
 2091 3170 3950 1021 2380  300  302  458 7750  157 4811 4930 5000 5090
 5160 5219 5410 5920 6025 6139 5040 6200 6201 6212   22   64   84  162
  164 8000  182  183  186  190 5326  192  194  199  200  208  258  264
 1431 1441 1455 1494 1496  284 1507 1540 1571 1724 1754 1757 1759 1779
 1803 1804 1806 1809 1818 1828 1830 1862 1864 1865 1984 2014 2019 2021
 2090 2188 2198 3180 3200 3245 3310 3380 3609 3610 3612 3613 3600 3629
 3630 3650 3780 3900 3934 3973 5143 3880 3882 3915 4001 4004 4011 4023
 4027 4051 4052 4053 4056 4057 4059 4060 4061 4063 4066 4071 4089 4092
 4105 4110 4111 4126 4137 4140 4180 4200 4240 4260 4309 4340 4360 4450
 4700  530  556  607  699  738  827 1022 1123 1133 2375 2415 2499 4068
 2951 2952 3602 4182 4490 1030  250  299  308  316  335  404  407  322
  472 

In [35]:
# Load dataset
df = pd.read_csv('../data/MachineLearningRating_v3_cleaned.csv')

# Segment by provinces (example: Gauteng vs Western Cape)
groupA_province = df[df['Province'] == 'Gauteng']['TotalClaims']
groupB_province = df[df['Province'] == 'Western Cape']['TotalClaims']


C:\Users\amanu\AppData\Local\Temp\ipykernel_9840\2380098913.py:2: DtypeWarning: Columns (45) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/MachineLearningRating_v3_cleaned.csv')


In [36]:
# Segment by zip codes (example: postal code 1001 vs 2001)
groupA_zip = df[df['PostalCode'] == 7560]['TotalClaims']
groupB_zip = df[df['PostalCode'] == 4067]['TotalClaims']


In [38]:
# Convert TotalPremium and TotalClaims to numeric (force non-numeric values to NaN)
df['TotalPremium'] = pd.to_numeric(df['TotalPremium'], errors='coerce')
df['TotalClaims'] = pd.to_numeric(df['TotalClaims'], errors='coerce')

# Optionally, handle NaN values (choose your method)
# You can fill NaNs with 0 or with the mean, depending on your case
df['TotalPremium'].fillna(0, inplace=True)
df['TotalClaims'].fillna(0, inplace=True)

# Now, perform the subtraction for margin differences
groupA_margin = df[df['PostalCode'] == 7560]['TotalPremium'] - df[df['PostalCode'] == 7560]['TotalClaims']
groupB_margin = df[df['PostalCode'] == 4067]['TotalPremium'] - df[df['PostalCode'] == 4067]['TotalClaims']


C:\Users\amanu\AppData\Local\Temp\ipykernel_9840\3591641343.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['TotalPremium'].fillna(0, inplace=True)
C:\Users\amanu\AppData\Local\Temp\ipykernel_9840\3591641343.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, wh

In [39]:
# Segment by gender for risk differences
groupA_gender = df[df['Gender'] == 'Female']['TotalClaims']
groupB_gender = df[df['Gender'] == 'Male']['TotalClaims']


### Statistical Testing

In [41]:
from scipy.stats import ttest_ind
import pandas as pd

# Convert relevant columns to numeric, forcing invalid parsing to NaN
df['TotalPremium'] = pd.to_numeric(df['TotalPremium'], errors='coerce')
df['TotalClaims'] = pd.to_numeric(df['TotalClaims'], errors='coerce')

# Handle NaN values (you can use different strategies like dropping or filling with mean/median)
df['TotalPremium'].fillna(0, inplace=True)
df['TotalClaims'].fillna(0, inplace=True)

# Calculate risk (you can define risk as the difference between TotalPremium and TotalClaims)
df['Risk'] = df['TotalPremium'] - df['TotalClaims']

# Segment by provinces (replace with actual province names)
groupA_province = df[df['Province'] == 'Gauteng']['Risk']
groupB_province = df[df['Province'] == 'Western Cape']['Risk']

# Ensure both groups are numeric (this should already be covered above but double-check)
groupA_province = pd.to_numeric(groupA_province, errors='coerce')
groupB_province = pd.to_numeric(groupB_province, errors='coerce')

# Perform the t-test for risk differences between provinces
t_stat_province, p_value_province = ttest_ind(groupA_province, groupB_province, nan_policy='omit')

print(f"p-value for risk differences across provinces: {p_value_province}")


C:\Users\amanu\AppData\Local\Temp\ipykernel_9840\123321415.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['TotalPremium'].fillna(0, inplace=True)
C:\Users\amanu\AppData\Local\Temp\ipykernel_9840\123321415.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, whe

p-value for risk differences across provinces: 0.15541895767348024


In [43]:
from scipy.stats import ttest_ind
import pandas as pd

# Convert 'TotalPremium' and 'TotalClaims' to numeric values, forcing invalid values to NaN
df['TotalPremium'] = pd.to_numeric(df['TotalPremium'], errors='coerce')
df['TotalClaims'] = pd.to_numeric(df['TotalClaims'], errors='coerce')

# Fill NaN values (here using 0 as an example, but you can use mean/median if preferred)
df['TotalPremium'].fillna(0, inplace=True)
df['TotalClaims'].fillna(0, inplace=True)

# Calculate the risk (difference between TotalPremium and TotalClaims)
df['Risk'] = df['TotalPremium'] - df['TotalClaims']

# Segment by zip codes (replace with actual zip codes)
groupA_zip = df[df['PostalCode'] == 7560]['Risk']
groupB_zip = df[df['PostalCode'] == 4067]['Risk']

# Ensure both groups are numeric (convert to numeric again to be sure)
groupA_zip = pd.to_numeric(groupA_zip, errors='coerce')
groupB_zip = pd.to_numeric(groupB_zip, errors='coerce')

# Handle NaN values in both groups by omitting them
groupA_zip = groupA_zip.dropna()
groupB_zip = groupB_zip.dropna()

# Perform the t-test for risk differences between zip codes
t_stat_zip, p_value_zip = ttest_ind(groupA_zip, groupB_zip, nan_policy='omit')

print(f"p-value for risk differences between zip codes: {p_value_zip}")


p-value for risk differences between zip codes: 0.0


C:\Users\amanu\AppData\Local\Temp\ipykernel_9840\1098636950.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['TotalPremium'].fillna(0, inplace=True)
C:\Users\amanu\AppData\Local\Temp\ipykernel_9840\1098636950.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, w

In [44]:
# Perform t-test for margin differences between zip codes
t_stat_margin, p_value_margin = ttest_ind(groupA_margin, groupB_margin)
print(f"p-value for margin differences between zip codes: {p_value_margin}")


p-value for margin differences between zip codes: 0.0


In [45]:
# Perform t-test for risk differences between genders
t_stat_gender, p_value_gender = ttest_ind(groupA_gender, groupB_gender)
print(f"p-value for risk differences between Women and Men: {p_value_gender}")


p-value for risk differences between Women and Men: 0.8041073961270343


### Analyze and Report

In [46]:
def analyze_p_value(p_value, hypothesis_name):
    if p_value < 0.05:
        print(f"Reject the null hypothesis for {hypothesis_name}.")
    else:
        print(f"Fail to reject the null hypothesis for {hypothesis_name}.")

# Analyze all hypotheses
analyze_p_value(p_value_province, "Risk Differences Across Provinces")
analyze_p_value(p_value_zip, "Risk Differences Between Zip Codes")
analyze_p_value(p_value_margin, "Margin Differences Between Zip Codes")
analyze_p_value(p_value_gender, "Risk Differences Between Women and Men")


Fail to reject the null hypothesis for Risk Differences Across Provinces.
Reject the null hypothesis for Risk Differences Between Zip Codes.
Reject the null hypothesis for Margin Differences Between Zip Codes.
Fail to reject the null hypothesis for Risk Differences Between Women and Men.


In [7]:
import pandas as pd
from scipy.stats import chi2_contingency

# Load your dataset
df = pd.read_csv('../data/MachineLearningRating_v3_cleaned.csv')

# Convert 'TotalClaims' to numeric, forcing errors to NaN and then fill NaNs with 0
df['TotalClaims'] = pd.to_numeric(df['TotalClaims'], errors='coerce').fillna(0)

# Create a new column to categorize whether a claim is made or not
df['ClaimMade'] = df['TotalClaims'] > 0

# Generate a contingency table for 'Province' and 'ClaimMade'
contingency_table_provinces = pd.crosstab(df['Province'], df['ClaimMade'])

print("Contingency Table for Provinces and ClaimMade:")
print(contingency_table_provinces)

# Perform Chi-Square test
chi2_provinces, p_provinces, dof_provinces, expected_provinces = chi2_contingency(contingency_table_provinces)

print(f"\nChi-Square Statistic for Provinces: {chi2_provinces}")
print(f"p-value: {p_provinces}")
print(f"Degrees of Freedom: {dof_provinces}")
print("Expected Frequencies:")
print(expected_provinces)

# Interpret the result
if p_provinces < 0.05:
    print("\nThere are significant risk differences across provinces (reject the null hypothesis).")
else:
    print("\nThere are no significant risk differences across provinces (fail to reject the null hypothesis).")


C:\Users\amanu\AppData\Local\Temp\ipykernel_18728\2138467552.py:5: DtypeWarning: Columns (45) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/MachineLearningRating_v3_cleaned.csv')


Contingency Table for Provinces and ClaimMade:
ClaimMade       False  True 
Province                    
Eastern Cape    30330      6
Free State       8088     11
Gauteng        392543   1322
KwaZulu-Natal  169298    483
Limpopo         24769     67
Mpumalanga      52590    128
North West     142938    349
Northern Cape    6372      8
Western Cape   170430    366

Chi-Square Statistic for Provinces: 167.08117743413868
p-value: 5.272438857224102e-32
Degrees of Freedom: 8
Expected Frequencies:
[[3.02528875e+04 8.31124950e+01]
 [8.07681091e+03 2.21890855e+01]
 [3.92785916e+05 1.07908435e+03]
 [1.69315846e+05 4.65154355e+02]
 [2.47679560e+04 6.80439717e+01]
 [5.25735668e+04 1.44433166e+02]
 [1.42894432e+05 3.92567908e+02]
 [6.36252051e+03 1.74794870e+01]
 [1.70328065e+05 4.67935182e+02]]

There are significant risk differences across provinces (reject the null hypothesis).


In [8]:
# Convert 'TotalClaims' to numeric, forcing errors to NaN and then fill NaNs with 0
df['TotalClaims'] = pd.to_numeric(df['TotalClaims'], errors='coerce').fillna(0)

# Create a new column to categorize whether a claim is made or not
df['ClaimMade'] = df['TotalClaims'] > 0

# For risk differences between zip codes, filter top zip codes
top_zip_codes = df['PostalCode'].value_counts().nlargest(5).index
df_top_zip = df[df['PostalCode'].isin(top_zip_codes)]

# Generate a contingency table for 'PostalCode' and 'ClaimMade'
contingency_table_zip = pd.crosstab(df_top_zip['PostalCode'], df_top_zip['ClaimMade'])

print("Contingency Table for Zip Codes and ClaimMade:")
print(contingency_table_zip)

# Perform Chi-Square test
chi2_zip, p_zip, dof_zip, expected_zip = chi2_contingency(contingency_table_zip)

print(f"\nChi-Square Statistic for Zip Codes: {chi2_zip}")
print(f"p-value: {p_zip}")
print(f"Degrees of Freedom: {dof_zip}")
print("Expected Frequencies:")
print(expected_zip)

# Interpret the result
if p_zip < 0.05:
    print("\nThere are significant risk differences between zip codes (reject the null hypothesis).")
else:
    print("\nThere are no significant risk differences between zip codes (fail to reject the null hypothesis).")

# For risk differences between gender
contingency_table_gender = pd.crosstab(df['Gender'], df['ClaimMade'])

print("Contingency Table for Gender and ClaimMade:")
print(contingency_table_gender)

# Perform Chi-Square test
chi2_gender, p_gender, dof_gender, expected_gender = chi2_contingency(contingency_table_gender)

print(f"\nChi-Square Statistic for Gender: {chi2_gender}")
print(f"p-value: {p_gender}")
print(f"Degrees of Freedom: {dof_gender}")
print("Expected Frequencies:")
print(expected_gender)

# Interpret the result
if p_gender < 0.05:
    print("\nThere are significant risk differences between Women and Men (reject the null hypothesis).")
else:
    print("\nThere are no significant risk differences between Women and Men (fail to reject the null hypothesis).")


Contingency Table for Zip Codes and ClaimMade:
ClaimMade    False  True 
PostalCode               
122          48961    210
299          25479     67
2000        133012    486
7405         18489     29
7784         28535     50

Chi-Square Statistic for Zip Codes: 60.22649731769842
p-value: 2.5997263446847924e-12
Degrees of Freedom: 4
Expected Frequencies:
[[4.90088415e+04 1.62158493e+02]
 [2.54617532e+04 8.42468294e+01]
 [1.33057744e+05 4.40256135e+02]
 [1.84569304e+04 6.10695525e+01]
 [2.84907310e+04 9.42689900e+01]]

There are significant risk differences between zip codes (reject the null hypothesis).
Contingency Table for Gender and ClaimMade:
ClaimMade       False  True 
Gender                      
Female           6741     14
Male            42723     94
Not specified  947894   2632

Chi-Square Statistic for Gender: 6.04191188477959
p-value: 0.048754589499511086
Degrees of Freedom: 2
Expected Frequencies:
[[6.73649311e+03 1.85068863e+01]
 [4.26996929e+04 1.17307084e+02]
 [9.47